In [ ]:
from Methods.base import Base, convert_strF_to_arrF, calculate_formula
from CPU import funcs
import pandas as pd
import os
import numpy as np

In [ ]:
Nam = 2024

FOLDER_CT = "DATA_CT/CT_MaxiMin"
PATH_DF_CT = f"{FOLDER_CT}/{Nam}.csv"
PATH_DATA = "HOSE_2007_2024.xlsx"
PATH_SAVE = "Data"
INTEREST = 1.06
VALUEARG_THRESHOLD = 500000000.0

os.makedirs(PATH_SAVE+f"/{Nam}", exist_ok=True)

df_CT = pd.read_csv(PATH_DF_CT)
data = pd.read_excel(PATH_DATA)

data = data[data["TIME"]>=2007]
data = data[data["TIME"]<=Nam].reset_index(drop=True)

In [ ]:
results = []
vis = Base(data, INTEREST, VALUEARG_THRESHOLD)

for i in range(len(df_CT)):
    ct_ = df_CT.loc[i, "CT"]
    list_value = [ct_]
    ct = convert_strF_to_arrF(ct_)
    weight = calculate_formula(ct, vis.OPERAND)

    # Max
    GeoMax, HarMax, ValGeo, GeoLim, ValHar, HarLim, GeoRank, HarRank = funcs.single_invest(
        weight, vis.INDEX, vis.PROFIT, vis.PROFIT_RANK, vis.PROFIT_RANK_NI, INTEREST
    )

    # Tinh CtyMax
    start, end, end2 = vis.INDEX[0], vis.INDEX[1], vis.INDEX[2]
    wgt = weight[start:end]
    arr_max = np.where(wgt==max(wgt))[0]
    if arr_max.shape[0] == 1:
        CtyMax = vis.symbol_name[vis.SYMBOL[arr_max[0]]]
        ProMax = vis.PROFIT[arr_max[0]]
        Value = weight[arr_max[0]]
    else:
        CtyMax = "NotInvest"
        ProMax = 1.06
        Value = -1.7976931348623157e+308
    
    list_value.extend([
        GeoMax, HarMax, Value, CtyMax, ProMax, ValGeo, GeoLim, ValHar, HarLim, GeoRank, HarRank
    ])

    # Nguong
    Nguong_, GeoNgn_, HarNgn_, ProNgn_ = funcs.multi_invest_1(
        weight, vis.INDEX, vis.PROFIT, INTEREST
    )

    arr_inv_idx = np.where(wgt > Nguong_)[0]
    arr_value = weight[arr_inv_idx]
    arr_symbol = vis.SYMBOL[arr_inv_idx]
    
    sort_ = np.argsort(arr_value)[::-1]
    arr_value = arr_value[sort_]
    arr_symbol = arr_symbol[sort_]
    
    CtyNgn = "_".join([vis.symbol_name[s] for s in arr_symbol])
    Values_Ngn = arr_value

    ProNgn1 = vis.PROFIT[start:end][wgt > Nguong_]
    if len(ProNgn1) > 0:
        ProNgn1 = ProNgn1.mean()
    else:
        ProNgn1 = vis.INTEREST
    list_value.extend([
        Nguong_, GeoNgn_, HarNgn_, ProNgn1, CtyNgn, Values_Ngn
    ])

    # Nguong2
    ValGeoNgn2, GeoNgn2, ValHarNgn2, HarNgn2 = funcs.multi_invest_2(
        weight, vis.INDEX, vis.PROFIT, vis.SYMBOL, INTEREST, vis.BOOL_ARG
    )
    bool_wgt = weight > HarNgn2
    bool_arg = vis.VALUEARG >= VALUEARG_THRESHOLD
    size = vis.INDEX.shape[0] - 1
    reason = 0

    for i in range(size-2, -1, -1):
        start, end = vis.INDEX[i], vis.INDEX[i+1]
        inv_cyc_val = bool_wgt[start:end] & bool_arg[start:end]
        if reason == 0:
            inv_cyc_sym = vis.SYMBOL[start:end]
            end2 = vis.INDEX[i+2]
            pre_cyc_val = bool_wgt[end:end2]
            pre_cyc_sym = vis.SYMBOL[end:end2]
            coms = np.intersect1d(pre_cyc_sym[pre_cyc_val], inv_cyc_sym[inv_cyc_val])
            isin = np.full(end-start, False)
            for j in range(end-start):
                if inv_cyc_sym[j] in coms:
                    isin[j] = True
            
            lst_com = np.where(isin)[0]
        else:
            lst_com = np.where(inv_cyc_val)[0]
        
        if len(lst_com) == 0:
            if np.count_nonzero(inv_cyc_val) == 0:
                reason = 1
        else:
            reason = 0
    
    arr_value = weight[lst_com]
    arr_symbol = vis.SYMBOL[lst_com]
    arr_profit_ = vis.PROFIT[lst_com]
    sort_ = np.argsort(arr_value)[::-1]
    arr_value = arr_value[sort_]
    arr_symbol = arr_symbol[sort_]

    CtyNgn2 = "_".join([vis.symbol_name[s] for s in arr_symbol])
    Values_Ngn2 = arr_value
    
    list_value.extend([
        ValGeoNgn2, GeoNgn2, ValHarNgn2, HarNgn2, arr_profit_.mean() if len(arr_profit_) > 0 else vis.INTEREST, CtyNgn2, Values_Ngn2
    ])

    # Slope
    list_value.extend(list(funcs.find_slope(weight, vis.INDEX, vis.PROFIT, vis.INTEREST)))

    # maximumMin

    ValGeo3, MinGeo3, ValHar3, MinHar3 = funcs.maximum_min_3(
        weight, vis.INDEX, vis.PROFIT, vis.INTEREST
    )
    list_value.extend([ValGeo3, MinGeo3, ValHar3, MinHar3])

    results.append(list_value)

data_save = pd.DataFrame(results)
data_save.columns = ["CT"]+"GeoMax, HarMax, Value, CtyMax, ProMax, ValGeo, GeoLim, ValHar, HarLim, GeoRank, HarRank".split(", ")\
                    +["Nguong1", "GeoNgn1", "HarNgn1", "ProNgn1", "CtyNgn1", "Values_Ngn1"]\
                    +["ValGeoNgn2", "GeoNgn2", "ValHarNgn2", "HarNgn2", "ProNgn2", "CtyNgn2", "Values_Ngn2"]\
                    +["Slope_avg", "Slope_wgt_avg"]\
                    +["ValGeo3", "MinGeo3", "ValHar3", "MinHar3"]
data_save.to_csv(PATH_SAVE+f"/{Nam}/Invest.csv", index=False)

In [ ]:
from collections import Counter
import json as js
with open(f"{PATH_SAVE}/{Nam}/Khuyen_nghi_dau_tu_theo_valueMax.json", "w") as f:
    js.dump(dict(sorted(dict(Counter(data_save["CtyMax"])).items(), key=lambda x: x[1], reverse=True)), f, indent=4)

dict_ = {"So cong thuc khong dau tu": len(data_save[data_save["CtyNgn2"]==""])}
temp = []
for i in range(len(data_save)):
    text = data_save.loc[i, "CtyNgn2"]
    lst = text.split("_")
    if "" in lst:
        lst.remove("")
    temp.extend(lst)
for k, v in dict(sorted(dict(Counter(temp)).items(), key=lambda x: x[1], reverse=True)).items():
    dict_[k] = v
with open(f"{PATH_SAVE}/{Nam}/Khuyen_nghi_dau_tu_theo_Nguong2.json", "w") as f:
    js.dump(dict_, f, indent=4)


dict_ = {"So cong thuc khong dau tu": len(data_save[data_save["CtyNgn1"]==""])}
temp = []
for i in range(len(data_save)):
    text = data_save.loc[i, "CtyNgn1"]
    lst = text.split("_")
    if "" in lst:
        lst.remove("")
    temp.extend(lst)
for k, v in dict(sorted(dict(Counter(temp)).items(), key=lambda x: x[1], reverse=True)).items():
    dict_[k] = v
with open(f"{PATH_SAVE}/{Nam}/Khuyen_nghi_dau_tu_theo_Nguong.json", "w") as f:
    js.dump(dict_, f, indent=4)